# Lab 09 — Hyperparameter Optimization with Optuna

 
> **주제:** Optuna를 이용한 체계적인 하이퍼파라미터 탐색

---

## 학습 목표

| # | 목표 |  
|---|---| 
| 1 | 하이퍼파라미터 탐색 전략 비교 (Grid / Random / Bayesian) |  
| 2 | Optuna 핵심 API: `Study`, `Trial`, `suggest_*` |  
| 3 | 선형 모델 (Ridge/Lasso) 튜닝 |  
| 4 | 분류 모델 (Logistic Regression, SVM) 튜닝 |  
| 5 | 트리 모델 (Random Forest, Gradient Boosting) 튜닝 |  
| 6 | 시각화 & 결과 분석 |  

---

**데이터셋:**
- 회귀: California Housing (sklearn) — 주택 가격 예측
- 분류: Breast Cancer Wisconsin (sklearn) — 암 진단 이진 분류

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)  # 불필요한 로그 숨김

from sklearn.datasets import fetch_california_housing, load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    mean_squared_error, r2_score,
    accuracy_score, roc_auc_score, classification_report
)
import time

# 한글 폰트
_fp = '/System/Library/Fonts/AppleGothic.ttf'
fm.fontManager.addfont(_fp)
plt.rcParams['font.family'] = fm.FontProperties(fname=_fp).get_name()
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style='whitegrid')

rng = np.random.default_rng(42)
print(f'Optuna version: {optuna.__version__}')

---
## Part 1. 하이퍼파라미터 탐색 전략

### 1-1. 왜 하이퍼파라미터 튜닝이 중요한가?

머신러닝 모델의 성능은 **하이퍼파라미터(Hyperparameter)** 에 크게 의존합니다.

| 구분 | 파라미터 | 하이퍼파라미터 |
|---|---|---|
| 정의 | 모델이 데이터에서 **학습**하는 값 | 학습 전에 **사람이 설정**하는 값 |
| 예시 | 신경망 가중치, 회귀 계수 | 학습률, 정규화 강도, 트리 깊이 |
| 최적화 방법 | Gradient Descent | 하이퍼파라미터 탐색 |

### 1-2. 세 가지 탐색 전략 비교

```
Grid Search:   모든 조합을 격자로 시도 → 완전 탐색, 비용 큼
Random Search: 무작위 조합 시도       → 효율적, 중요한 파라미터에 집중
Bayesian Opt:  이전 결과를 학습하며 탐색 → 가장 효율적, Optuna가 사용
```

In [ ]:
# 세 가지 탐색 전략 시각화
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 공통 설정
n_grid = 5
n_random = 25

# 가상의 손실 함수 (2D)
def loss_surface(x, y):
    return (np.sin(2*x) + np.cos(2*y) + 0.5*(x-0.3)**2 + 0.5*(y-0.7)**2)

x_range = np.linspace(0, 1, 200)
y_range = np.linspace(0, 1, 200)
XX, YY  = np.meshgrid(x_range, y_range)
ZZ      = loss_surface(XX, YY)

titles = ['Grid Search\n(25개 고정 격자)', 'Random Search\n(25개 무작위)', 'Bayesian (Optuna)\n(25개, 이전 결과 활용)']
colors_strat = ['steelblue', 'tomato', 'seagreen']

for i, (ax, title, color) in enumerate(zip(axes, titles, colors_strat)):
    ax.contourf(XX, YY, ZZ, levels=20, cmap='RdYlGn_r', alpha=0.7)
    ax.contour(XX, YY, ZZ, levels=10, colors='white', alpha=0.3, linewidths=0.5)

    if i == 0:  # Grid Search
        xs = np.linspace(0.1, 0.9, n_grid)
        ys = np.linspace(0.1, 0.9, n_grid)
        Xg, Yg = np.meshgrid(xs, ys)
        ax.scatter(Xg.ravel(), Yg.ravel(), c=color, s=60, zorder=5, edgecolors='white')
    elif i == 1:  # Random Search
        np.random.seed(0)
        rx = np.random.uniform(0, 1, n_random)
        ry = np.random.uniform(0, 1, n_random)
        ax.scatter(rx, ry, c=color, s=60, zorder=5, edgecolors='white')
    else:  # Bayesian (TPE 시뮬레이션)
        np.random.seed(1)
        bx, by = [0.5], [0.5]
        for _ in range(24):
            best_x = bx[np.argmin([loss_surface(x, y) for x, y in zip(bx, by)])]
            best_y = by[np.argmin([loss_surface(x, y) for x, y in zip(bx, by)])]
            # 최적점 근처 + 탐색 균형
            if np.random.rand() < 0.6:
                nx = np.clip(best_x + np.random.randn()*0.15, 0, 1)
                ny = np.clip(best_y + np.random.randn()*0.15, 0, 1)
            else:
                nx = np.random.uniform(0, 1)
                ny = np.random.uniform(0, 1)
            bx.append(nx); by.append(ny)
        ax.scatter(bx, by, c=color, s=60, zorder=5, edgecolors='white')
        # 최적점 강조
        best_i = np.argmin([loss_surface(x, y) for x, y in zip(bx, by)])
        ax.scatter(bx[best_i], by[best_i], c='gold', s=200, zorder=6,
                   edgecolors='black', linewidths=2, marker='*')

    ax.set_title(title, fontsize=11)
    ax.set_xlabel('하이퍼파라미터 1')
    ax.set_ylabel('하이퍼파라미터 2')

plt.suptitle('하이퍼파라미터 탐색 전략 비교 (색상: 손실 크기, 초록=낮음/좋음)', y=1.02)
plt.tight_layout()
plt.show()

print('Grid Search:  모든 격자점 평가 → 고비용, 중요 파라미터 없이 균등 분배')
print('Random Search: 균등 분포 무작위 → 중요 파라미터 축에 더 다양한 값 탐색')
print('Bayesian:     이전 결과 학습 → 유망한 영역 집중 탐색 (★ 금색=최적점 근처)')

### 1-3. 탐색 비용 비교

| 전략 | 탐색 횟수 (파라미터 3개, 각 5값) | 특징 |
|---|---|---|
| **Grid Search** | $5^3 = 125$ | 조합 폭발 문제 |
| **Random Search** | 사용자 지정 (예: 50) | 빠르고 효율적 |
| **Bayesian (Optuna)** | 사용자 지정 (예: 50) | 가장 효율적, 연속 공간 탐색 가능 |

> 파라미터가 10개라면? Grid: $5^{10} = 9,765,625$번 vs Optuna: 100번으로 충분히 좋은 결과!

In [ ]:
# Grid Search vs Random Search vs Optuna 속도 비교 (간단한 예제)
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform, loguniform

# 데이터 준비
housing = fetch_california_housing(as_frame=True)
X_house = housing.data.values.astype(np.float32)
y_house = housing.target.values.astype(np.float32)
X_tr_h, X_te_h, y_tr_h, y_te_h = train_test_split(
    X_house, y_house, test_size=0.2, random_state=42
)
scaler_h = StandardScaler()
X_tr_h_s = scaler_h.fit_transform(X_tr_h)
X_te_h_s  = scaler_h.transform(X_te_h)

# 1) Grid Search
grid_params = {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}
t0 = time.time()
gs = GridSearchCV(Ridge(), grid_params, cv=3, scoring='r2')
gs.fit(X_tr_h_s, y_tr_h)
t_grid = time.time() - t0
best_gs_alpha = gs.best_params_['alpha']
ridge_gs = Ridge(alpha=best_gs_alpha).fit(X_tr_h_s, y_tr_h)
r2_gs = r2_score(y_te_h, ridge_gs.predict(X_te_h_s))

# 2) Random Search
rand_params = {'alpha': loguniform(0.001, 100)}
t0 = time.time()
rs = RandomizedSearchCV(Ridge(), rand_params, n_iter=20, cv=3, scoring='r2', random_state=42)
rs.fit(X_tr_h_s, y_tr_h)
t_rand = time.time() - t0
best_rs_alpha = rs.best_params_['alpha']
ridge_rs = Ridge(alpha=best_rs_alpha).fit(X_tr_h_s, y_tr_h)
r2_rs = r2_score(y_te_h, ridge_rs.predict(X_te_h_s))

# 3) Optuna
def ridge_objective(trial):
    alpha = trial.suggest_float('alpha', 1e-3, 100.0, log=True)
    model = Ridge(alpha=alpha)
    scores = cross_val_score(model, X_tr_h_s, y_tr_h, cv=3, scoring='r2')
    return scores.mean()

t0 = time.time()
study_ridge = optuna.create_study(direction='maximize')
study_ridge.optimize(ridge_objective, n_trials=20)
t_optuna = time.time() - t0
best_optuna_alpha = study_ridge.best_params['alpha']
ridge_opt = Ridge(alpha=best_optuna_alpha).fit(X_tr_h_s, y_tr_h)
r2_opt = r2_score(y_te_h, ridge_opt.predict(X_te_h_s))

print(f'{'방법':<20} {'최적 alpha':>12} {'Test R²':>10} {'소요 시간':>10}')
print('-' * 56)
print(f'{'Grid Search':<20} {best_gs_alpha:>12.4f} {r2_gs:>10.4f} {t_grid:>8.2f}s')
print(f'{'Random Search':<20} {best_rs_alpha:>12.4f} {r2_rs:>10.4f} {t_rand:>8.2f}s')
print(f'{'Optuna (Bayesian)':<20} {best_optuna_alpha:>12.4f} {r2_opt:>10.4f} {t_optuna:>8.2f}s')

---
## Part 2. Optuna 핵심 API

### 2-1. 세 가지 핵심 개념

```
Study   — 하나의 최적화 실험 (목적함수 + 탐색 방향 정의)
Trial   — Study 내의 하나의 시도 (하이퍼파라미터 샘플 하나)
suggest_* — Trial에서 하이퍼파라미터 값을 제안받는 메서드
```

### 2-2. `suggest_*` API 정리

| 메서드 | 용도 | 예시 |
|---|---|---|
| `suggest_float(name, low, high)` | 연속 실수 | 학습률, 드롭아웃 |
| `suggest_float(name, low, high, log=True)` | 로그 스케일 실수 | 정규화 강도 (1e-5 ~ 1) |
| `suggest_int(name, low, high)` | 정수 | 트리 깊이, 뉴런 수 |
| `suggest_categorical(name, choices)` | 범주형 | 활성화 함수, 알고리즘 |

### 2-3. 기본 Optuna 패턴

```python
def objective(trial):                          # 1. 목적함수 정의
    param = trial.suggest_float('lr', 1e-4, 1e-1, log=True)  # 2. 파라미터 제안
    # ... 모델 학습 ...
    return validation_score                    # 3. 최적화할 값 반환

study = optuna.create_study(direction='maximize')  # 4. Study 생성
study.optimize(objective, n_trials=50)             # 5. 최적화 실행
print(study.best_params)                           # 6. 최적 파라미터 확인
```

In [ ]:
# Optuna 기본 사용법 — 간단한 수학 함수 최적화
# 목표: f(x, y) = -(x-2)^2 - (y+1)^2 의 최댓값 찾기 (실제 최대: x=2, y=-1)

def simple_objective(trial):
    x = trial.suggest_float('x', -5, 5)    # x: [-5, 5] 연속값
    y = trial.suggest_float('y', -5, 5)    # y: [-5, 5] 연속값
    return -(x - 2)**2 - (y + 1)**2        # 최대화 (최적: x=2, y=-1)

study_simple = optuna.create_study(direction='maximize')  # 최대화
study_simple.optimize(simple_objective, n_trials=50)

print('=== Optuna 기본 예제 ===' )
print(f'최적 파라미터: {study_simple.best_params}')
print(f'최적 값:       {study_simple.best_value:.4f}')
print(f'총 시도 횟수:  {len(study_simple.trials)}')
print(f'\n실제 최적: x=2.0, y=-1.0  →  최대값: 0.0')

# 탐색 과정 시각화
trial_nums = [t.number for t in study_simple.trials]
trial_vals = [t.value  for t in study_simple.trials]
best_so_far = np.maximum.accumulate(trial_vals)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 수렴 곡선
axes[0].plot(trial_nums, trial_vals, 'o', alpha=0.5, color='steelblue', ms=4, label='개별 Trial')
axes[0].plot(trial_nums, best_so_far, '-', color='tomato', lw=2.5, label='현재까지 최고값')
axes[0].set_title('최적화 수렴 곡선')
axes[0].set_xlabel('Trial 번호')
axes[0].set_ylabel('목적함수 값')
axes[0].legend()

# 파라미터 탐색 분포
xs = [t.params['x'] for t in study_simple.trials]
ys = [t.params['y'] for t in study_simple.trials]
sc = axes[1].scatter(xs, ys, c=trial_vals, cmap='RdYlGn', s=50, zorder=3)
axes[1].scatter(2, -1, c='red', s=200, marker='*', zorder=5, label='실제 최적점')
axes[1].set_title('파라미터 탐색 공간 (색=목적함수 값)')
axes[1].set_xlabel('x'); axes[1].set_ylabel('y')
axes[1].legend()
plt.colorbar(sc, ax=axes[1])

# Trial별 파라미터 히스토리
axes[2].scatter(trial_nums, xs, alpha=0.7, label='x', color='steelblue', s=20)
axes[2].scatter(trial_nums, ys, alpha=0.7, label='y', color='tomato', s=20)
axes[2].axhline(2,  color='steelblue', lw=1.5, linestyle='--', label='x 최적값(2)')
axes[2].axhline(-1, color='tomato',    lw=1.5, linestyle='--', label='y 최적값(-1)')
axes[2].set_title('Trial에 따른 파라미터 수렴')
axes[2].set_xlabel('Trial 번호')
axes[2].set_ylabel('파라미터 값')
axes[2].legend(fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
# suggest_* 종류별 사용법 데모
def demo_suggest_api(trial):
    # 연속 실수 (선형 스케일)
    dropout   = trial.suggest_float('dropout', 0.0, 0.5)

    # 연속 실수 (로그 스케일) — 학습률처럼 자릿수가 중요한 경우
    lr        = trial.suggest_float('lr', 1e-5, 1e-1, log=True)

    # 정수
    n_layers  = trial.suggest_int('n_layers', 1, 4)
    hidden    = trial.suggest_int('hidden', 32, 512, step=32)  # 32의 배수

    # 범주형
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'sgd', 'rmsprop'])

    # 조건부 파라미터 (optimizer가 sgd일 때만 momentum 사용)
    if optimizer == 'sgd':
        momentum = trial.suggest_float('momentum', 0.0, 0.99)
    else:
        momentum = 0.0

    return dropout + lr * 100  # 의미없는 목적함수 (데모용)

study_demo = optuna.create_study(direction='maximize')
study_demo.optimize(demo_suggest_api, n_trials=5)

print('=== suggest_* API 데모 — 샘플 5개 ===' )
print(f'{'Trial':>6} | {'dropout':>8} | {'lr':>10} | {'n_layers':>8} | {'hidden':>7} | {'optimizer':>10}')
print('-' * 65)
for t in study_demo.trials:
    p = t.params
    print(f"{t.number:>6} | {p.get('dropout', 0):>8.3f} | {p.get('lr', 0):>10.2e} | "
          f"{p.get('n_layers', 0):>8} | {p.get('hidden', 0):>7} | {p.get('optimizer', ''):>10}")

---
## Part 3. 선형 모델 튜닝 — Ridge & Lasso

### 3-1. 실습 개요

**California Housing 데이터**로 주택 가격을 예측합니다.

| 모델 | 튜닝할 파라미터 | 탐색 범위 |
|---|---|---|
| **Ridge** | `alpha` (L2 정규화 강도) | 로그 스케일 [1e-3, 1e3] |
| **Lasso** | `alpha` (L1 정규화 강도) | 로그 스케일 [1e-4, 10.0] |

> **핵심 아이디어:** alpha가 클수록 정규화가 강해져 모델이 단순해집니다.
> Optuna는 최적의 alpha를 자동으로 찾아줍니다.

In [ ]:
# 데이터 준비 (회귀)
housing = fetch_california_housing(as_frame=True)
X_reg = housing.data.values.astype(np.float32)
y_reg = housing.target.values.astype(np.float32)

X_tr_r, X_te_r, y_tr_r, y_te_r = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)
scaler_r = StandardScaler()
X_tr_rs  = scaler_r.fit_transform(X_tr_r)
X_te_rs  = scaler_r.transform(X_te_r)

print(f'California Housing 데이터')
print(f'  특성 수  : {X_reg.shape[1]}')
print(f'  Train  : {len(X_tr_r)}개')
print(f'  Test   : {len(X_te_r)}개')
print(f'  목표값  : 중위 주택 가격 (×$100K)  범위 [{y_reg.min():.2f}, {y_reg.max():.2f}]')

In [ ]:
# Ridge 회귀 — Optuna 튜닝
def ridge_objective(trial):
    alpha = trial.suggest_float('alpha', 1e-3, 1e3, log=True)
    model = Ridge(alpha=alpha)
    # 3-fold 교차 검증으로 안정적인 성능 측정
    scores = cross_val_score(model, X_tr_rs, y_tr_r, cv=3, scoring='r2')
    return scores.mean()

print('Ridge 최적화 시작...')
study_ridge = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=42)  # TPE: Tree-structured Parzen Estimator
)
study_ridge.optimize(ridge_objective, n_trials=40)

best_ridge_alpha = study_ridge.best_params['alpha']
print(f'\n최적 alpha: {best_ridge_alpha:.4f}')
print(f'CV R² (최적): {study_ridge.best_value:.4f}')

# 최적 파라미터로 최종 모델 학습
ridge_final = Ridge(alpha=best_ridge_alpha)
ridge_final.fit(X_tr_rs, y_tr_r)
ridge_test_r2  = r2_score(y_te_r, ridge_final.predict(X_te_rs))
ridge_test_mse = mean_squared_error(y_te_r, ridge_final.predict(X_te_rs))
print(f'Test  R²: {ridge_test_r2:.4f}')
print(f'Test MSE: {ridge_test_mse:.4f}')

In [ ]:
# Lasso 회귀 — Optuna 튜닝
def lasso_objective(trial):
    alpha     = trial.suggest_float('alpha', 1e-4, 10.0, log=True)
    max_iter  = trial.suggest_int('max_iter', 500, 3000, step=500)
    model = Lasso(alpha=alpha, max_iter=max_iter)
    scores = cross_val_score(model, X_tr_rs, y_tr_r, cv=3, scoring='r2')
    return scores.mean()

print('Lasso 최적화 시작...')
study_lasso = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=42)
)
study_lasso.optimize(lasso_objective, n_trials=40)

best_lasso_params = study_lasso.best_params
print(f'\n최적 파라미터: {best_lasso_params}')
print(f'CV R² (최적): {study_lasso.best_value:.4f}')

lasso_final = Lasso(**best_lasso_params)
lasso_final.fit(X_tr_rs, y_tr_r)
lasso_test_r2  = r2_score(y_te_r, lasso_final.predict(X_te_rs))
lasso_test_mse = mean_squared_error(y_te_r, lasso_final.predict(X_te_rs))
print(f'Test  R²: {lasso_test_r2:.4f}')
print(f'Test MSE: {lasso_test_mse:.4f}')

# Lasso 특성 선택 효과 확인
nonzero = np.sum(np.abs(lasso_final.coef_) > 1e-6)
print(f'\nLasso 비영 계수 수: {nonzero}/{len(lasso_final.coef_)} (나머지는 0으로 축소됨)')

In [ ]:
# Ridge vs Lasso 비교 시각화
fig, axes = plt.subplots(2, 3, figsize=(15, 9))

# Ridge: alpha 탐색 과정
r_alphas = [t.params['alpha'] for t in study_ridge.trials]
r_vals   = [t.value for t in study_ridge.trials]
best_so_far_r = np.maximum.accumulate(r_vals)

axes[0, 0].scatter(r_alphas, r_vals, alpha=0.6, color='steelblue', s=40)
axes[0, 0].axvline(best_ridge_alpha, color='tomato', lw=2, linestyle='--',
                    label=f'최적 α={best_ridge_alpha:.3f}')
axes[0, 0].set_xscale('log')
axes[0, 0].set_title('Ridge — alpha 탐색')
axes[0, 0].set_xlabel('alpha (로그 스케일)')
axes[0, 0].set_ylabel('CV R²')
axes[0, 0].legend()

# Lasso: alpha 탐색 과정
l_alphas = [t.params['alpha'] for t in study_lasso.trials]
l_vals   = [t.value for t in study_lasso.trials]

axes[0, 1].scatter(l_alphas, l_vals, alpha=0.6, color='seagreen', s=40)
axes[0, 1].axvline(best_lasso_params['alpha'], color='tomato', lw=2, linestyle='--',
                    label=f"최적 α={best_lasso_params['alpha']:.4f}")
axes[0, 1].set_xscale('log')
axes[0, 1].set_title('Lasso — alpha 탐색')
axes[0, 1].set_xlabel('alpha (로그 스케일)')
axes[0, 1].set_ylabel('CV R²')
axes[0, 1].legend()

# 수렴 곡선 비교
best_so_far_l = np.maximum.accumulate(l_vals)
axes[0, 2].plot(best_so_far_r, color='steelblue', lw=2, label='Ridge')
axes[0, 2].plot(best_so_far_l, color='seagreen', lw=2, label='Lasso')
axes[0, 2].set_title('최적화 수렴 곡선 비교')
axes[0, 2].set_xlabel('Trial 번호')
axes[0, 2].set_ylabel('Best CV R² (누적)')
axes[0, 2].legend()

# 가중치 비교
feat_names = housing.feature_names
x_pos = np.arange(len(feat_names))
w = 0.35
axes[1, 0].bar(x_pos - w/2, ridge_final.coef_, w, label='Ridge', color='steelblue', alpha=0.85)
axes[1, 0].bar(x_pos + w/2, lasso_final.coef_, w, label='Lasso', color='seagreen', alpha=0.85)
axes[1, 0].axhline(0, color='black', lw=0.8)
axes[1, 0].set_xticks(x_pos)
axes[1, 0].set_xticklabels(feat_names, rotation=30, ha='right', fontsize=9)
axes[1, 0].set_title('학습된 계수 비교\n(Lasso는 일부 계수를 0으로 만듦)')
axes[1, 0].legend()

# 예측 vs 실제 (Ridge)
y_pred_ridge = ridge_final.predict(X_te_rs)
axes[1, 1].scatter(y_te_r, y_pred_ridge, alpha=0.2, s=8, color='steelblue')
lo, hi = y_te_r.min(), y_te_r.max()
axes[1, 1].plot([lo, hi], [lo, hi], 'r--', lw=2)
axes[1, 1].set_title(f'Ridge 예측 vs 실제\nR²={ridge_test_r2:.4f}')
axes[1, 1].set_xlabel('실제값'); axes[1, 1].set_ylabel('예측값')

# 예측 vs 실제 (Lasso)
y_pred_lasso = lasso_final.predict(X_te_rs)
axes[1, 2].scatter(y_te_r, y_pred_lasso, alpha=0.2, s=8, color='seagreen')
axes[1, 2].plot([lo, hi], [lo, hi], 'r--', lw=2)
axes[1, 2].set_title(f'Lasso 예측 vs 실제\nR²={lasso_test_r2:.4f}')
axes[1, 2].set_xlabel('실제값'); axes[1, 2].set_ylabel('예측값')

plt.suptitle('Ridge vs Lasso — Optuna 튜닝 결과', fontsize=13, y=1.01)
plt.tight_layout()
plt.show()

---
## Part 4. 분류 모델 튜닝

### 4-1. 실습 개요

**Breast Cancer Wisconsin 데이터**로 유방암 진단을 분류합니다.

| 모델 | 튜닝할 파라미터 | 탐색 범위 |
|---|---|---|
| **Logistic Regression** | `C` (역 정규화 강도), `solver` | C: 로그 [1e-3, 100], solver: 범주형 |
| **SVM** | `C`, `kernel`, `gamma` | C: 로그, kernel: 범주형, gamma: 로그 |

> **팁:** 분류에서는 `accuracy`, `roc_auc`, `f1` 등 다양한 메트릭을 목적함수로 사용할 수 있습니다.

In [ ]:
# 데이터 준비 (분류)
cancer = load_breast_cancer(as_frame=True)
X_cls = cancer.data.values.astype(np.float32)
y_cls = cancer.target.values

X_tr_c, X_te_c, y_tr_c, y_te_c = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=42, stratify=y_cls
)
scaler_c = StandardScaler()
X_tr_cs  = scaler_c.fit_transform(X_tr_c)
X_te_cs  = scaler_c.transform(X_te_c)

print(f'Breast Cancer Wisconsin 데이터')
print(f'  특성 수: {X_cls.shape[1]} (세포핵 특성들)')
print(f'  클래스: Malignant(악성)=0 / Benign(양성)=1')
print(f'  Train  : {len(X_tr_c)}개  (양성={y_tr_c.sum()}, 악성={len(y_tr_c)-y_tr_c.sum()})')
print(f'  Test   : {len(X_te_c)}개  (양성={y_te_c.sum()}, 악성={len(y_te_c)-y_te_c.sum()})')

In [ ]:
# Logistic Regression — Optuna 튜닝
# 목적함수: ROC-AUC (불균형 데이터에 더 적합)

def lr_objective(trial):
    C      = trial.suggest_float('C', 1e-3, 100.0, log=True)
    solver = trial.suggest_categorical('solver', ['lbfgs', 'liblinear', 'saga'])

    # solver별 지원 패널티 처리 (조건부 파라미터)
    if solver == 'liblinear':
        penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    else:
        penalty = 'l2'

    model = LogisticRegression(
        C=C, solver=solver, penalty=penalty, max_iter=1000, random_state=42
    )
    scores = cross_val_score(model, X_tr_cs, y_tr_c, cv=5, scoring='roc_auc')
    return scores.mean()

print('Logistic Regression 최적화 시작...')
study_lr = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=42)
)
study_lr.optimize(lr_objective, n_trials=50)

best_lr_params = study_lr.best_params
print(f'\n최적 파라미터: {best_lr_params}')
print(f'CV ROC-AUC (최적): {study_lr.best_value:.4f}')

# 최적 파라미터로 최종 학습
C_opt    = best_lr_params['C']
sol_opt  = best_lr_params['solver']
pen_opt  = best_lr_params.get('penalty', 'l2')
lr_final = LogisticRegression(C=C_opt, solver=sol_opt, penalty=pen_opt,
                               max_iter=1000, random_state=42)
lr_final.fit(X_tr_cs, y_tr_c)

y_pred_lr  = lr_final.predict(X_te_cs)
y_prob_lr  = lr_final.predict_proba(X_te_cs)[:, 1]
print(f'Test Accuracy: {accuracy_score(y_te_c, y_pred_lr):.4f}')
print(f'Test ROC-AUC : {roc_auc_score(y_te_c, y_prob_lr):.4f}')

In [ ]:
# SVM — Optuna 튜닝 (kernel별 파라미터 조건부 제안)

def svm_objective(trial):
    C      = trial.suggest_float('C', 1e-2, 100.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])

    # kernel별 추가 파라미터
    kwargs = {}
    if kernel in ['rbf', 'poly']:
        kwargs['gamma'] = trial.suggest_float('gamma', 1e-4, 1.0, log=True)
    if kernel == 'poly':
        kwargs['degree'] = trial.suggest_int('degree', 2, 5)

    model = SVC(C=C, kernel=kernel, probability=True, random_state=42, **kwargs)
    scores = cross_val_score(model, X_tr_cs, y_tr_c, cv=5, scoring='roc_auc')
    return scores.mean()

print('SVM 최적화 시작...')
study_svm = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=42)
)
study_svm.optimize(svm_objective, n_trials=40)

best_svm_params = study_svm.best_params
print(f'\n최적 파라미터: {best_svm_params}')
print(f'CV ROC-AUC (최적): {study_svm.best_value:.4f}')

svm_final = SVC(
    C=best_svm_params['C'],
    kernel=best_svm_params['kernel'],
    gamma=best_svm_params.get('gamma', 'scale'),
    degree=best_svm_params.get('degree', 3),
    probability=True, random_state=42
)
svm_final.fit(X_tr_cs, y_tr_c)

y_pred_svm = svm_final.predict(X_te_cs)
y_prob_svm = svm_final.predict_proba(X_te_cs)[:, 1]
print(f'Test Accuracy: {accuracy_score(y_te_c, y_pred_svm):.4f}')
print(f'Test ROC-AUC : {roc_auc_score(y_te_c, y_prob_svm):.4f}')

In [ ]:
# 분류 모델 결과 시각화
from sklearn.metrics import roc_curve, confusion_matrix

fig, axes = plt.subplots(2, 2, figsize=(13, 10))

# 수렴 곡선 비교
lr_best_curve  = np.maximum.accumulate([t.value for t in study_lr.trials])
svm_best_curve = np.maximum.accumulate([t.value for t in study_svm.trials])
axes[0, 0].plot(lr_best_curve,  lw=2, color='steelblue', label=f'Logistic Reg (최종: {study_lr.best_value:.4f})')
axes[0, 0].plot(svm_best_curve, lw=2, color='tomato',    label=f'SVM (최종: {study_svm.best_value:.4f})')
axes[0, 0].set_title('최적화 수렴 곡선 (ROC-AUC)')
axes[0, 0].set_xlabel('Trial 번호')
axes[0, 0].set_ylabel('Best CV ROC-AUC')
axes[0, 0].legend()

# ROC Curve
fpr_lr, tpr_lr, _ = roc_curve(y_te_c, y_prob_lr)
fpr_sv, tpr_sv, _ = roc_curve(y_te_c, y_prob_svm)
axes[0, 1].plot(fpr_lr, tpr_lr, lw=2, color='steelblue',
                label=f'Logistic Reg AUC={roc_auc_score(y_te_c, y_prob_lr):.4f}')
axes[0, 1].plot(fpr_sv, tpr_sv, lw=2, color='tomato',
                label=f'SVM AUC={roc_auc_score(y_te_c, y_prob_svm):.4f}')
axes[0, 1].plot([0, 1], [0, 1], 'k--', lw=1, label='무작위 분류기')
axes[0, 1].set_title('ROC 곡선')
axes[0, 1].set_xlabel('FPR (위양성률)')
axes[0, 1].set_ylabel('TPR (재현율)')
axes[0, 1].legend()

# 혼동 행렬 (LR)
cm_lr = confusion_matrix(y_te_c, y_pred_lr)
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', ax=axes[1, 0],
            xticklabels=['악성', '양성'], yticklabels=['악성', '양성'])
axes[1, 0].set_title(f'Logistic Regression 혼동 행렬\nAcc={accuracy_score(y_te_c, y_pred_lr):.4f}')
axes[1, 0].set_xlabel('예측'); axes[1, 0].set_ylabel('실제')

# 혼동 행렬 (SVM)
cm_svm = confusion_matrix(y_te_c, y_pred_svm)
sns.heatmap(cm_svm, annot=True, fmt='d', cmap='Reds', ax=axes[1, 1],
            xticklabels=['악성', '양성'], yticklabels=['악성', '양성'])
axes[1, 1].set_title(f'SVM 혼동 행렬\nAcc={accuracy_score(y_te_c, y_pred_svm):.4f}')
axes[1, 1].set_xlabel('예측'); axes[1, 1].set_ylabel('실제')

plt.suptitle('분류 모델 Optuna 튜닝 결과', fontsize=13, y=1.01)
plt.tight_layout()
plt.show()

# SVM kernel 탐색 분포
kernels = [t.params.get('kernel', 'N/A') for t in study_svm.trials]
from collections import Counter
kernel_cnt = Counter(kernels)
print('\nSVM 탐색된 kernel 분포:')
for k, v in kernel_cnt.items():
    print(f'  {k}: {v}회')

---
## Part 5. 트리 모델 튜닝

### 5-1. 실습 개요

트리 모델은 튜닝할 파라미터가 많아 Optuna가 특히 효과적입니다.

| 모델 | 주요 하이퍼파라미터 |
|---|---|
| **Random Forest** | `n_estimators`, `max_depth`, `min_samples_split`, `max_features` |
| **Gradient Boosting** | `n_estimators`, `learning_rate`, `max_depth`, `subsample`, `min_samples_leaf` |

In [ ]:
# Random Forest — Optuna 튜닝 (분류)

def rf_objective(trial):
    n_estimators      = trial.suggest_int('n_estimators', 50, 300, step=50)
    max_depth         = trial.suggest_int('max_depth', 3, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf  = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features      = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42, n_jobs=-1
    )
    scores = cross_val_score(model, X_tr_cs, y_tr_c, cv=5, scoring='roc_auc')
    return scores.mean()

print('Random Forest 최적화 시작...')
study_rf = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=42)
)
study_rf.optimize(rf_objective, n_trials=50)

print(f'\n최적 파라미터:')
for k, v in study_rf.best_params.items():
    print(f'  {k}: {v}')
print(f'CV ROC-AUC (최적): {study_rf.best_value:.4f}')

# 최종 학습
rf_final = RandomForestClassifier(**study_rf.best_params, random_state=42, n_jobs=-1)
rf_final.fit(X_tr_cs, y_tr_c)
y_pred_rf = rf_final.predict(X_te_cs)
y_prob_rf = rf_final.predict_proba(X_te_cs)[:, 1]
print(f'Test Accuracy: {accuracy_score(y_te_c, y_pred_rf):.4f}')
print(f'Test ROC-AUC : {roc_auc_score(y_te_c, y_prob_rf):.4f}')

In [ ]:
# Gradient Boosting — Optuna 튜닝

def gb_objective(trial):
    n_estimators   = trial.suggest_int('n_estimators', 50, 300, step=50)
    learning_rate  = trial.suggest_float('learning_rate', 1e-3, 0.3, log=True)
    max_depth      = trial.suggest_int('max_depth', 2, 8)
    subsample      = trial.suggest_float('subsample', 0.5, 1.0)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)

    model = GradientBoostingClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    scores = cross_val_score(model, X_tr_cs, y_tr_c, cv=5, scoring='roc_auc')
    return scores.mean()

print('Gradient Boosting 최적화 시작...')
study_gb = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=42)
)
study_gb.optimize(gb_objective, n_trials=50)

print(f'\n최적 파라미터:')
for k, v in study_gb.best_params.items():
    print(f'  {k}: {v}')
print(f'CV ROC-AUC (최적): {study_gb.best_value:.4f}')

gb_final = GradientBoostingClassifier(**study_gb.best_params, random_state=42)
gb_final.fit(X_tr_cs, y_tr_c)
y_pred_gb = gb_final.predict(X_te_cs)
y_prob_gb = gb_final.predict_proba(X_te_cs)[:, 1]
print(f'Test Accuracy: {accuracy_score(y_te_c, y_pred_gb):.4f}')
print(f'Test ROC-AUC : {roc_auc_score(y_te_c, y_prob_gb):.4f}')

In [ ]:
# 트리 모델 파라미터 중요도 시각화 (Optuna built-in)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Random Forest 파라미터 중요도
try:
    importances_rf = optuna.importance.get_param_importances(study_rf)
    params_rf = list(importances_rf.keys())
    imps_rf   = list(importances_rf.values())
    sorted_idx_rf = np.argsort(imps_rf)
    axes[0].barh([params_rf[i] for i in sorted_idx_rf],
                  [imps_rf[i]  for i in sorted_idx_rf],
                  color='steelblue', edgecolor='k', alpha=0.85)
    axes[0].set_title('Random Forest\n하이퍼파라미터 중요도 (Optuna)')
    axes[0].set_xlabel('중요도 (FAnova)')
except Exception:
    axes[0].text(0.5, 0.5, 'FAnova 분석 실패\n(Trial 수 부족)', ha='center', va='center',
                  transform=axes[0].transAxes)

# Gradient Boosting 파라미터 중요도
try:
    importances_gb = optuna.importance.get_param_importances(study_gb)
    params_gb = list(importances_gb.keys())
    imps_gb   = list(importances_gb.values())
    sorted_idx_gb = np.argsort(imps_gb)
    axes[1].barh([params_gb[i] for i in sorted_idx_gb],
                  [imps_gb[i]  for i in sorted_idx_gb],
                  color='tomato', edgecolor='k', alpha=0.85)
    axes[1].set_title('Gradient Boosting\n하이퍼파라미터 중요도 (Optuna)')
    axes[1].set_xlabel('중요도 (FAnova)')
except Exception:
    axes[1].text(0.5, 0.5, 'FAnova 분석 실패\n(Trial 수 부족)', ha='center', va='center',
                  transform=axes[1].transAxes)

plt.suptitle('어떤 하이퍼파라미터가 성능에 가장 큰 영향을 미치나?', fontsize=12, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Gradient Boosting: 주요 파라미터 탐색 공간 시각화
gb_trials = study_gb.trials
gb_lr_vals   = [t.params['learning_rate']   for t in gb_trials]
gb_depth_vals = [t.params['max_depth']      for t in gb_trials]
gb_sub_vals  = [t.params['subsample']       for t in gb_trials]
gb_scores    = [t.value for t in gb_trials]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# learning_rate vs score
sc1 = axes[0].scatter(gb_lr_vals, gb_scores, c=gb_scores, cmap='RdYlGn', s=50, alpha=0.8)
axes[0].axvline(study_gb.best_params['learning_rate'], color='red', lw=2, linestyle='--',
                 label=f"최적 lr={study_gb.best_params['learning_rate']:.4f}")
axes[0].set_xscale('log')
axes[0].set_title('learning_rate vs CV ROC-AUC')
axes[0].set_xlabel('learning_rate (로그 스케일)')
axes[0].set_ylabel('CV ROC-AUC')
axes[0].legend(fontsize=8)
plt.colorbar(sc1, ax=axes[0])

# max_depth vs score
sc2 = axes[1].scatter(gb_depth_vals, gb_scores, c=gb_scores, cmap='RdYlGn', s=50, alpha=0.8)
axes[1].axvline(study_gb.best_params['max_depth'], color='red', lw=2, linestyle='--',
                 label=f"최적 depth={study_gb.best_params['max_depth']}")
axes[1].set_title('max_depth vs CV ROC-AUC')
axes[1].set_xlabel('max_depth')
axes[1].set_ylabel('CV ROC-AUC')
axes[1].legend(fontsize=8)
plt.colorbar(sc2, ax=axes[1])

# subsample vs score
sc3 = axes[2].scatter(gb_sub_vals, gb_scores, c=gb_scores, cmap='RdYlGn', s=50, alpha=0.8)
axes[2].axvline(study_gb.best_params['subsample'], color='red', lw=2, linestyle='--',
                 label=f"최적 subsample={study_gb.best_params['subsample']:.3f}")
axes[2].set_title('subsample vs CV ROC-AUC')
axes[2].set_xlabel('subsample')
axes[2].set_ylabel('CV ROC-AUC')
axes[2].legend(fontsize=8)
plt.colorbar(sc3, ax=axes[2])

plt.suptitle('Gradient Boosting 하이퍼파라미터 탐색 공간 시각화', y=1.02)
plt.tight_layout()
plt.show()

---
## Part 6. 전체 결과 분석 및 시각화

### 6-1. 모델별 성능 최종 비교

In [ ]:
# 모든 모델 최종 성능 비교
from sklearn.dummy import DummyClassifier

# 기준선: 무작위 분류기
dummy = DummyClassifier(strategy='stratified', random_state=42)
dummy.fit(X_tr_cs, y_tr_c)
y_prob_dummy = dummy.predict_proba(X_te_cs)[:, 1]
y_pred_dummy = dummy.predict(X_te_cs)

model_results = {
    '기준선 (무작위)':           (accuracy_score(y_te_c, y_pred_dummy),
                                   roc_auc_score(y_te_c, y_prob_dummy)),
    'Logistic Reg (튜닝 전)':    None,  # 아래서 계산
    'Logistic Reg (Optuna)':     (accuracy_score(y_te_c, y_pred_lr),
                                   roc_auc_score(y_te_c, y_prob_lr)),
    'SVM (Optuna)':              (accuracy_score(y_te_c, y_pred_svm),
                                   roc_auc_score(y_te_c, y_prob_svm)),
    'Random Forest (Optuna)':    (accuracy_score(y_te_c, y_pred_rf),
                                   roc_auc_score(y_te_c, y_prob_rf)),
    'Gradient Boosting (Optuna)':(accuracy_score(y_te_c, y_pred_gb),
                                   roc_auc_score(y_te_c, y_prob_gb)),
}

# 튜닝 전 Logistic Regression (기본값)
lr_default = LogisticRegression(random_state=42, max_iter=1000)
lr_default.fit(X_tr_cs, y_tr_c)
y_pred_lrd = lr_default.predict(X_te_cs)
y_prob_lrd = lr_default.predict_proba(X_te_cs)[:, 1]
model_results['Logistic Reg (튜닝 전)'] = (
    accuracy_score(y_te_c, y_pred_lrd),
    roc_auc_score(y_te_c, y_prob_lrd)
)

# 정렬: ROC-AUC 기준
sorted_models = sorted(model_results.items(), key=lambda x: x[1][1])

print(f'{'모델':<30} {'Accuracy':>10} {'ROC-AUC':>10}')
print('-' * 54)
for name, (acc, auc) in sorted_models:
    marker = ' ←' if '기준선' in name else ('★' if auc == max(v[1] for v in model_results.values()) else '')
    print(f'{name:<30} {acc:>10.4f} {auc:>10.4f} {marker}')

In [ ]:
# 최종 비교 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

names   = [n for n, _ in sorted_models]
accs    = [v[0] for _, v in sorted_models]
aucs    = [v[1] for _, v in sorted_models]
colors_final = ['gray'] + ['#aed6f1'] + ['steelblue', 'tomato', 'seagreen', 'darkorange']

# Accuracy 비교
bars_acc = axes[0].barh(names, accs, color=colors_final, edgecolor='k', alpha=0.85)
for bar, val in zip(bars_acc, accs):
    axes[0].text(val + 0.001, bar.get_y() + bar.get_height()/2,
                  f'{val:.4f}', va='center', fontsize=9)
axes[0].set_title('모델별 Test Accuracy 비교')
axes[0].set_xlabel('Accuracy')
axes[0].set_xlim(0.85, 1.02)
axes[0].axvline(accs[0], color='gray', lw=1.5, linestyle=':', label='기준선')

# ROC-AUC 비교
bars_auc = axes[1].barh(names, aucs, color=colors_final, edgecolor='k', alpha=0.85)
for bar, val in zip(bars_auc, aucs):
    axes[1].text(val + 0.001, bar.get_y() + bar.get_height()/2,
                  f'{val:.4f}', va='center', fontsize=9)
axes[1].set_title('모델별 Test ROC-AUC 비교')
axes[1].set_xlabel('ROC-AUC')
axes[1].set_xlim(0.85, 1.02)
axes[1].axvline(aucs[0], color='gray', lw=1.5, linestyle=':', label='기준선')

plt.suptitle('Optuna 튜닝 후 모델 성능 비교 (Breast Cancer)', fontsize=13, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# 튜닝 전후 비교 (Logistic Regression 기준)
print('=== 튜닝 효과 분석: Logistic Regression ===')
print(f'{'':25} {'기본값':>10} {'Optuna':>10} {'향상':>8}')
print('-' * 58)
acc_before = accuracy_score(y_te_c, y_pred_lrd)
auc_before = roc_auc_score(y_te_c, y_prob_lrd)
acc_after  = accuracy_score(y_te_c, y_pred_lr)
auc_after  = roc_auc_score(y_te_c, y_prob_lr)
print(f'{'Test Accuracy':25} {acc_before:>10.4f} {acc_after:>10.4f} {acc_after-acc_before:>+8.4f}')
print(f'{'Test ROC-AUC':25} {auc_before:>10.4f} {auc_after:>10.4f} {auc_after-auc_before:>+8.4f}')

print(f'\n기본값 파라미터: C=1.0, solver=lbfgs, penalty=l2')
print(f'최적 파라미터: {study_lr.best_params}')

# Optuna 최적화 요약 출력
print('\n=== Optuna Study 요약 ===')
all_studies = [
    ('Ridge (회귀)', study_ridge, 'R²'),
    ('Lasso (회귀)', study_lasso, 'R²'),
    ('Logistic Reg', study_lr, 'ROC-AUC'),
    ('SVM',          study_svm, 'ROC-AUC'),
    ('Random Forest', study_rf, 'ROC-AUC'),
    ('Grad Boosting', study_gb, 'ROC-AUC'),
]
print(f'{'모델':<20} {'Trial 수':>8} {'최적값':>10} {'메트릭':>10}')
print('-' * 52)
for name, study, metric in all_studies:
    print(f'{name:<20} {len(study.trials):>8} {study.best_value:>10.4f} {metric:>10}')

---
## Exercise

### Exercise 1. 기본 Optuna 패턴 구현

아래 코드를 완성하여 **Ridge 회귀**의 `alpha`를 Optuna로 튜닝하세요.

- 탐색 범위: `alpha` ∈ [0.001, 100] (로그 스케일)
- Trial 수: 30
- 평가 메트릭: 5-fold CV R²
- 방향: maximize

최적 alpha 값과 Test R²를 출력하세요.

In [ ]:
# Exercise 1: Ridge 튜닝
def ex1_objective(trial):
    # Your code here: alpha 파라미터 제안
    pass

# Your code here: Study 생성 및 최적화

# Your code here: 최적 파라미터로 최종 학습 및 Test R² 출력

### Exercise 2. 조건부 파라미터 구현

아래 목적함수를 완성하세요. `kernel` 선택에 따라 다른 파라미터를 제안합니다.

- `kernel='linear'` 이면: `C` 만 튜닝 (범위: [0.01, 100], 로그)
- `kernel='rbf'` 이면: `C` + `gamma` 튜닝 (gamma 범위: [1e-4, 1.0], 로그)
- `kernel='poly'` 이면: `C` + `gamma` + `degree` 튜닝 (degree: 정수 [2, 5])

30 Trial 후 최적 파라미터와 Test Accuracy를 출력하고,  
어떤 kernel이 가장 많이 선택되었는지 확인하세요.

In [ ]:
# Exercise 2: SVM 조건부 파라미터
def ex2_svm_objective(trial):
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    C      = trial.suggest_float('C', 0.01, 100.0, log=True)

    # Your code here: 조건부 파라미터 추가

    # Your code here: 모델 생성 및 CV 평가 반환
    pass

# Your code here: 최적화 실행 및 결과 출력

### Exercise 3. Random Forest 튜닝 및 기본값 비교

아래를 구현하세요:

1. **기본값 Random Forest** (파라미터 튜닝 없음) 학습 및 성능 기록
2. **Optuna로 튜닝된 Random Forest** 학습 (n_trials=30)
   - 파라미터: `n_estimators` [50,300], `max_depth` [3,15], `min_samples_leaf` [1,10]
3. 튜닝 전후 Test Accuracy와 ROC-AUC 비교 출력
4. 수렴 곡선 시각화 (x: Trial 번호, y: Best CV ROC-AUC)

In [ ]:
# Exercise 3: Random Forest 튜닝 전후 비교

# 1. 기본값 Random Forest
# Your code here

# 2. Optuna 튜닝
def ex3_rf_objective(trial):
    # Your code here
    pass

# Your code here: 최적화 실행

# 3. 비교 출력
# Your code here

# 4. 수렴 곡선
# Your code here

---
## Summary

| 개념 | 핵심 내용 |
|---|---|
| **하이퍼파라미터** | 학습 전에 사람이 설정하는 값 (학습률, 정규화 강도, 트리 깊이 등) |
| **Grid Search** | 모든 격자 조합 탐색 — 파라미터 증가 시 비용 폭발 |
| **Random Search** | 무작위 탐색 — Grid보다 효율적, 중요 파라미터에 더 다양한 값 |
| **Bayesian Opt** | 이전 결과를 학습해 유망한 영역 집중 탐색 |
| **Optuna Study** | 하나의 최적화 실험 — `direction` 설정 필수 |
| **Optuna Trial** | 하나의 하이퍼파라미터 조합 시도 |
| **suggest_float** | 연속 실수 제안, `log=True`로 로그 스케일 |
| **suggest_int** | 정수 제안, `step`으로 간격 설정 |
| **suggest_categorical** | 범주형 선택 (알고리즘, 활성화 함수 등) |
| **조건부 파라미터** | if문으로 파라미터 의존성 처리 (kernel별 gamma 등) |
| **파라미터 중요도** | `optuna.importance.get_param_importances()` |
| **TPE Sampler** | Tree-structured Parzen Estimator — Optuna 기본 알고리즘 |

---

**다음 강의 (Week 10):** 딥러닝 — 다층 퍼셉트론 (MLP), 역전파, 활성화 함수